### Analyze blocked reactions for ThermoModels under specific regression conditions

In [1]:
import os.path as path
import glob, os
from datetime import datetime
from importlib.metadata import version
import cobra
import thermo_flux
from thermo_flux.io import load_excel as ex
from thermo_flux.core.model import ThermoModel
from equilibrator_api import  Q_
import pandas as pd
from thermo_flux.io import helper_load as hl
import numpy as np
from thermo_flux.io import load_excel as ex
from scripts.logger import write_to_log
import gurobipy as gp
from gurobipy import GRB
from scripts.gen_model import gen_model
from scripts.gen_model import apply_physio_data
from scripts.gen_model import constrain_bounds_fva
from scripts.reaction_utils import list_blocked_reactions
from scripts.reaction_utils import count_blocked_pathways
from thermo_flux.solver.gurobi import compute_IIS

In [2]:
INPUT_MODEL = "datafiles/model.xlsx"
INPUT_KEGGS = "datafiles/ecoli_kegg_id.csv"
INPUT_REED = "regression/reed.csv"
INPUT_INCHI = "regression/InChIs.csv"
INPUT_GAMS = "regression/model_Ecoli_from-gams.xlsx"
INPUT_EXP_DATA = "regression/allPhysioData_formatted_forGSM_20230831.csv"
INPUT_EXP_CONC = "regression/allConcRange_20230912.csv"
INPUT_METABOLOMICS = "regression/metabolomics-Kochanowski_20230925.csv"

MODEL_NAME = "ecoli"

OUTPUT_DIR = "output"
OUTPUT_NAME = f"blocked_reactions"
OUTPUT_LOG = f"{OUTPUT_DIR}{path.sep}{OUTPUT_NAME}_log.txt"

CONDITIONS_TO_REGRESS = ["WT-Glc_I", "WT-Gal_I", "WT-Fruc_I", "WT-Mann_I", "dptsG-Glc_I", 
                         "WT-Ace_I", "WT-Succ_I", "WT-Fum_I", "WT-Glyc_I", "WT-Pyr_I",
                         "WT-GlyCAA_II"]

INCLUDE_CO2 = True
INCLUDE_O2 = True
ALLOW_OTHER_EXCRETION = False
RELAX_EXP_FLUX_BOUNDS = 2.0

time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
write_to_log(OUTPUT_LOG, f"Started analysis at: {time}", "w")

# Write package versions:
modules = ["pandas", "numpy", "equilibrator_api", "cobra"]
write_to_log(OUTPUT_LOG, f"Package versions used:")
versions_packages = [f"  {m}: {version(m)}\n" for m in modules]
write_to_log(OUTPUT_LOG, "".join(versions_packages))

In [5]:
tmodel = gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, OUTPUT_LOG, True, True)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-02
['Parameters', 'Exchange reactions', 'Reactions', 'Biomass Composition', 'Transmembrane reactions', 'Metabolites', 'references', 'Transmembrane_reactions_reed', 'Transmembrane reactions_Orth', 'Transmembrane reactions old', 'Sheet3', 'log', 'subsystems']
*** Reading data from Reactions ***
unknown metabolite '2dhglcn[c]' created
unknown metabolite 'nadh[c]' created
unknown metabolite 'glcn[c]' created
unknown metabolite 'nad[c]' created
unknown metabolite 'nadph[c]' created
unknown metabolite 'nadp[c]' created
unknown metabolite '2dhguln[c]' created
unknown metabolite 'idon-L[c]' created
unknown metabolite '3hcinnm[c]' created
unknown metabolite 'o2[c]' created
unknown metabolite 'dhcinnm[c]' created
unknown metabolite 'h2o[c]' created
unknown metabolite '3hpppn[c]' created
unknown metabolite 'dhpppn[c]' created
unknown metabolite 'phthr[c]' created
unknown metabolite '4hthr[c]' created
unknown m

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


*** Updating metabolite information ***
2dhglcn_c NOTHING DONE!
nadh_c NOTHING DONE!
glcn_c NOTHING DONE!
nad_c NOTHING DONE!
nadph_c NOTHING DONE!
nadp_c NOTHING DONE!
2dhguln_c NOTHING DONE!
idon-L_c NOTHING DONE!
3hcinnm_c NOTHING DONE!
o2_c NOTHING DONE!
dhcinnm_c NOTHING DONE!
h2o_c NOTHING DONE!
3hpppn_c NOTHING DONE!
dhpppn_c NOTHING DONE!
phthr_c NOTHING DONE!
4hthr_c NOTHING DONE!
pi_c NOTHING DONE!
5dglcn_c NOTHING DONE!
ru5p-D_c NOTHING DONE!
ara5p_c NOTHING DONE!
ACP_c NOTHING DONE!
atp_c NOTHING DONE!
ttdca_c NOTHING DONE!
amp_c NOTHING DONE!
myrsACP_c NOTHING DONE!
ppi_c NOTHING DONE!
ttdcea_c NOTHING DONE!
tdeACP_c NOTHING DONE!
hdca_c NOTHING DONE!
palmACP_c NOTHING DONE!
hdcea_c NOTHING DONE!
hdeACP_c NOTHING DONE!
ocdcea_c NOTHING DONE!
octeACP_c NOTHING DONE!
dtdp4aaddg_c NOTHING DONE!
unagamu_c NOTHING DONE!
dtdp_c NOTHING DONE!
unagamuf_c NOTHING DONE!
arbt6p_c NOTHING DONE!
g6p_c NOTHING DONE!
hqn_c NOTHING DONE!
4abut_c NOTHING DONE!
akg_c NOTHING DONE!
glu-L_c N

In [ ]:
tmodel_p = gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, OUTPUT_LOG, True, True)
tmodel_p = apply_physio_data(tmodel_p, CONDITIONS_TO_REGRESS[1], INPUT_EXP_DATA, INPUT_EXP_CONC, INPUT_METABOLOMICS, INPUT_GAMS, RELAX_EXP_FLUX_BOUNDS, INCLUDE_CO2, INCLUDE_O2, ALLOW_OTHER_EXCRETION, OUTPUT_LOG)

blocked_p = list_blocked_reactions(tmodel_p, CONDITIONS_TO_REGRESS[0], OUTPUT_LOG)
print(len(blocked_p))
# Keep SERASr as model becomes infeasible after removing it
to_keep = ["SERASr"]
blocked_p = [x for x in blocked_p if x not in to_keep]
print(len(blocked_p))

tmodel_p.remove_reactions(blocked_p)
for rxn in tmodel_p.reactions:
    thermo_flux.tools.drg_tools.reaction_balance(rxn, balance_charge=True, balance_mg=False)
tmodel_p.update_thermo_info(fit_unknown_dfG0=True)

tmodel_p.m = None  
tmodel_p.objective = tmodel_p.reactions.biomass_EX  
tmodel_p.add_TFBA_variables() 

tmodel_p.m.Params.TimeLimit = 3600
tmodel_p.m.Params.Threads = 16
tmodel_p.m.optimize() 

In [3]:
tmodel_p = gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, OUTPUT_LOG, True, True)
tmodel_p = apply_physio_data(tmodel_p, CONDITIONS_TO_REGRESS[0], INPUT_EXP_DATA, INPUT_EXP_CONC, INPUT_METABOLOMICS, INPUT_GAMS, RELAX_EXP_FLUX_BOUNDS, INCLUDE_CO2, INCLUDE_O2, ALLOW_OTHER_EXCRETION, OUTPUT_LOG)
#tmodel_p = constrain_bounds_fva(tmodel_p, OUTPUT_LOG) # comparison

Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-02
['Parameters', 'Exchange reactions', 'Reactions', 'Biomass Composition', 'Transmembrane reactions', 'Metabolites', 'references', 'Transmembrane_reactions_reed', 'Transmembrane reactions_Orth', 'Transmembrane reactions old', 'Sheet3', 'log', 'subsystems']
*** Reading data from Reactions ***
unknown metabolite '2dhglcn[c]' created
unknown metabolite 'nadh[c]' created
unknown metabolite 'glcn[c]' created
unknown metabolite 'nad[c]' created
unknown metabolite 'nadph[c]' created
unknown metabolite 'nadp[c]' created
unknown metabolite '2dhguln[c]' created
unknown metabolite 'idon-L[c]' created
unknown metabolite '3hcinnm[c]' created
unknown metabolite 'o2[c]' created
unknown metabolite 'dhcinnm[c]' created
unknown metabolite 'h2o[c]' created
unknown metabolite '3hpppn[c]' created
unknown metabolite 'dhpppn[c]' created
unknown metabolite 'phthr[c]' created
unknown metabolite '4hthr[c]' created
unknown m

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


2dhglcn_c NOTHING DONE!
nadh_c NOTHING DONE!
glcn_c NOTHING DONE!
nad_c NOTHING DONE!
nadph_c NOTHING DONE!
nadp_c NOTHING DONE!
2dhguln_c NOTHING DONE!
idon-L_c NOTHING DONE!
3hcinnm_c NOTHING DONE!
o2_c NOTHING DONE!
dhcinnm_c NOTHING DONE!
h2o_c NOTHING DONE!
3hpppn_c NOTHING DONE!
dhpppn_c NOTHING DONE!
phthr_c NOTHING DONE!
4hthr_c NOTHING DONE!
pi_c NOTHING DONE!
5dglcn_c NOTHING DONE!
ru5p-D_c NOTHING DONE!
ara5p_c NOTHING DONE!
ACP_c NOTHING DONE!
atp_c NOTHING DONE!
ttdca_c NOTHING DONE!
amp_c NOTHING DONE!
myrsACP_c NOTHING DONE!
ppi_c NOTHING DONE!
ttdcea_c NOTHING DONE!
tdeACP_c NOTHING DONE!
hdca_c NOTHING DONE!
palmACP_c NOTHING DONE!
hdcea_c NOTHING DONE!
hdeACP_c NOTHING DONE!
ocdcea_c NOTHING DONE!
octeACP_c NOTHING DONE!
dtdp4aaddg_c NOTHING DONE!
unagamu_c NOTHING DONE!
dtdp_c NOTHING DONE!
unagamuf_c NOTHING DONE!
arbt6p_c NOTHING DONE!
g6p_c NOTHING DONE!
hqn_c NOTHING DONE!
4abut_c NOTHING DONE!
akg_c NOTHING DONE!
glu-L_c NOTHING DONE!
sucsal_c NOTHING DONE!
4abu

In [4]:
blocked_p = list_blocked_reactions(tmodel_p, CONDITIONS_TO_REGRESS[0], OUTPUT_LOG)
print(len(blocked_p))
# Keep SERASr as model becomes infeasible after removing it
to_keep = ["SERASr"]
blocked_p = [x for x in blocked_p if x not in to_keep]
print(len(blocked_p))

['3HCINNMH', '3HPPPNH', '4HTHRS', '5DGLCNR', 'AADDGT', 'AB6PGH', 'ACACCT', 'ACACT1r', 'ACBIPGT', 'ACGAMT', 'ACMAMUT', 'ACNML', 'ACONMT', 'ACPS1', 'ADHEr', 'ADK4', 'ADNCYC', 'ADOCBIK', 'ADOCBLS', 'AGDC', 'AGMT', 'AHC', 'AHCYSNS', 'ALCD19', 'ALDD19x', 'ALLTAH', 'alltn', 'ALTRH', 'AMANAPE', 'AMANK', 'AMAOTr', 'AMMQT82', 'AMPMS', 'AOXSr', 'AP4AH', 'AP5AH', 'ARAI', 'ARGDC', 'BTS2', 'BUTCT', 'CBIAT', 'CBLAT', 'CDPMEK', 'CHRPL', 'CINNDO', 'CPPPGO', 'CRNBTCT', 'CRNCBCT', 'CRNCDH', 'CYANST', 'CYNTAH', 'DBTSr', 'DDGALK', 'DDGLK', 'DDPGALA', 'DHBD', 'DHBSr', 'DHCIND', 'DHCINDO', 'DHNAOT', 'DHPPD', 'DHPTDC', 'DMATT', 'DMQMT', 'DOGULNR', 'DXPRIi', 'DXPS', 'DXYLK', 'E4PD', 'ECAPEC', 'EDTXS3', 'EDTXS4', 'ENTCS', 'FAO3', 'FAO4', 'FCI', 'FCLK', 'FCLPA', 'FCLT', 'FFSD', 'FHL', 'G1PTT', 'G1SATi', 'GALCTD', 'GALCTND', 'GALKr', 'GALS3', 'GDMANE', 'GLCRAL', 'GLCRD', 'GLTPD', 'glucys', 'GLUTRR', 'GLUTRS', 'GMAND', 'GOFUCR', 'GP4GH', 'GPDDA1', 'GPDDA3', 'GPDDA5', 'GRTT', 'GTHOr', 'GTHS', 'GUI1', 'GUI2', 'HBZO

In [6]:
blocked = list_blocked_reactions(tmodel, "BASE", OUTPUT_LOG)

['4HTHRS', 'AADDGT', 'AB6PGH', 'ACBIPGT', 'ACGAMT', 'ACMAMUT', 'ACONMT', 'ACPS1', 'ADK4', 'ADNCYC', 'ADOCBIK', 'ADOCBLS', 'AHC', 'AHCYSNS', 'ALDD19x', 'AMAOTr', 'AMMQT82', 'AMPMS', 'AOXSr', 'AP4AH', 'AP5AH', 'BTS2', 'CBIAT', 'CBLAT', 'CDPMEK', 'CINNDO', 'CPPPGO', 'CRNBTCT', 'CRNCBCT', 'CRNCDH', 'CYANST', 'DBTSr', 'DHBD', 'DHBSr', 'DHCIND', 'DHNAOT', 'DHPTDC', 'DMATT', 'DMQMT', 'DOGULNR', 'DXPRIi', 'DXPS', 'DXYLK', 'E4PD', 'ECAPEC', 'EDTXS3', 'EDTXS4', 'ENTCS', 'FCLT', 'FHL', 'G1PTT', 'G1SATi', 'GDMANE', 'glucys', 'GLUTRR', 'GLUTRS', 'GMAND', 'GOFUCR', 'GP4GH', 'GPDDA1', 'GPDDA3', 'GPDDA5', 'GRTT', 'GTHOr', 'GTHS', 'HBZOPT', 'HEMEOS', 'HETZK', 'HMBS', 'HMPK1', 'ICHORSi', 'ICHORT', 'KG6PDC', 'MAN1PT2', 'MECDPDH', 'MECDPS', 'MEPCT', 'MI1PP', 'NNDMBRT', 'NPHS', 'OCTDPS', 'OHPBAT', 'OHPHM', 'OMBZLM', 'OMMBLHX', 'OMPHHX', 'OPHBDC', 'OPHHX', 'OXGDC2', 'PACCOAL', 'PDX5PO', 'PDX5PS', 'PEAMNO', 'PERD', 'PGLYCP', 'PLIPA3', 'PMANM', 'PMPK', 'PPBNGS', 'PPPGO', 'RHCCE', 'RZ5PP', 'SELNPS', 'SERASr', 

In [12]:
# Try to find out which reaction is causing the model to become infeasible
reaction_feasibility = dict()

for b in blocked_p:
    #tmodel = gen_model(MODEL_NAME, INPUT_MODEL, INPUT_KEGGS, INPUT_REED, INPUT_INCHI, INPUT_GAMS, OUTPUT_LOG, True, True)
    tmodel_p.remove_reactions([b])
    for rxn in tmodel_p.reactions:
        thermo_flux.tools.drg_tools.reaction_balance(rxn, balance_charge=True, balance_mg=False)
    tmodel_p.update_thermo_info(fit_unknown_dfG0=True)
        
    tmodel_p.m = None  
    tmodel_p.objective = tmodel_p.reactions.biomass_EX  
    tmodel_p.add_TFBA_variables() 

    tmodel_p.m.Params.TimeLimit = 120        
    tmodel_p.m.Params.Threads = 16           
    tmodel_p.m.Params.Method = 0             

    tmodel_p.m.Params.MIPFocus = 0           
    tmodel_p.m.Params.Heuristics = 0.05      
    tmodel_p.m.Params.Cuts = 0               
    tmodel_p.m.Params.Presolve = 1           

    tmodel_p.m.Params.LogToConsole = 0
    tmodel_p.m.Params.OutputFlag = 0

    tmodel_p.m.optimize()

    if tmodel_p.m.Status == 2 or tmodel_p.m.Status == 9:
        print(f"Removing reaction {b} remains feasible")
        reaction_feasibility[b] = 1
    elif tmodel_p.m.Status == 3 or tmodel_p.m.Status == 4:
        print(f"Removing reaction {b} causes infeasibility")
        reaction_feasibility[b] = 0



Identifying compounds...
[████████████████████████████████████████] 777/777 Mg_e                                                                             

Estimating dfG0'...
[████████████████████████████████████████] 777/777 Mg_e                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

Estimating drG0'...
[████████████████████████████████████████] 1065/1065 EX_oro                                                                                                                                                                                                          

KeyboardInterrupt: 

In [ ]:
count_blocked_pathways(blocked, "ecoli", "BASE", INPUT_MODEL)

In [ ]:
diff = list(set(blocked_p) - set(blocked))
print(diff)

In [ ]:
# Update reactions and stoichometric matrices
tmodel.remove_reactions(blocked)
for rxn in tmodel.reactions:
    thermo_flux.tools.drg_tools.reaction_balance(rxn, balance_charge=True, balance_mg=False)
tmodel.update_thermo_info(fit_unknown_dfG0=True)

In [ ]:
count_blocked_pathways(blocked, "ecoli", "BASE", INPUT_MODEL)

In [ ]:
tmodel.m = None  
tmodel.objective = tmodel.reactions.biomass_EX  
tmodel.add_TFBA_variables() 

In [ ]:
tmodel.m.Params.TimeLimit = 3600
tmodel.m.Params.Threads = 16    
compute_IIS(tmodel)

In [5]:
tmodel_p.remove_reactions(blocked_p)
for rxn in tmodel_p.reactions:
    thermo_flux.tools.drg_tools.reaction_balance(rxn, balance_charge=True, balance_mg=False)
tmodel_p.update_thermo_info(fit_unknown_dfG0=True)

Identifying compounds...
[████████████████████████████████████████] 777/777 Mg_e                                                                             

Estimating dfG0'...
[████████████████████████████████████████] 777/777 Mg_e                                                                                                                                                                                                                                     

Estimating drG0'...
[████████████████████████████████████████] 624/624 EX_oro                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [6]:

tmodel_p.m = None  
tmodel_p.objective = tmodel_p.reactions.biomass_EX  
tmodel_p.add_TFBA_variables() 

Set parameter NonConvex to value 2
Set parameter TimeLimit to value 10


In [7]:
tmodel_p.m.Params.TimeLimit = 3600
tmodel_p.m.Params.Threads = 16
tmodel_p.m.optimize() 
#compute_IIS(tmodel_p)

Set parameter TimeLimit to value 3600
Set parameter Threads to value 16
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7086 rows, 5506 columns and 186873 nonzeros
Model fingerprint: 0x2928f9f2
Model has 1 general constraint
Variable types: 4882 continuous, 624 integer (624 binary)
Coefficient statistics:
  Matrix range     [3e-06, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e-01, 1e+08]
  RHS range        [2e-14, 1e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 3754 rows and 2802 columns
Presolve time: 0.46s
Presolved: 3333 rows, 3063 columns, 168075 nonzeros
Presolved model has 359 quadratic constraint(s)
Variable types: 2557 continuous, 506 integer (506 binary)



In [8]:
tmodel_p.m.write("solutions/ecoli_WT_Glc_I_SOLUTION.sol")

In [ ]:
# Thermodynamic FVA vs normal


In [ ]:
disconnected = [m.id for m in tmodel.metabolites if len(m.reactions) == 0]
print("Disconnected metabolites:", disconnected)

In [ ]:
tmodel_regressed = apply_physio_data(tmodel, "WT-Glc_I", INPUT_EXP_DATA, INPUT_EXP_CONC, INPUT_METABOLOMICS, INPUT_GAMS, RELAX_EXP_FLUX_BOUNDS, INCLUDE_CO2, INCLUDE_O2, ALLOW_OTHER_EXCRETION, OUTPUT_LOG)

In [ ]:
blocked = list_blocked_reactions(tmodel_regressed, "WT-Glc_I", OUTPUT_LOG)

In [ ]:
count_blocked_pathways(blocked, "ecoli", "WT-Glc_I", INPUT_MODEL)

In [ ]:
rxns_df = pd.read_excel(INPUT_MODEL, sheet_name="Reactions")
rxns_df.columns = rxns_df.columns.str.strip()

blocked_ids = blocked 
blocked_info = rxns_df[rxns_df["Abbrevation"].isin(blocked_ids)]

nit_all = rxns_df[rxns_df["Subsystem"].str.contains("nitrogen", case=False, na=False)]
nit_blocked = blocked_info[blocked_info["Subsystem"].str.contains("nitrogen", case=False, na=False)]

print("Nitrogen total reactions:", len(nit_all))
print("Nitrogen blocked reactions:", len(nit_blocked))
print("Fraction blocked:", len(nit_blocked) / max(1, len(nit_all)))


In [ ]:
disconnected = [m.id for m in tmodel.metabolites if len(m.reactions) == 0]
print("Disconnected metabolites:", disconnected)